In [4]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd


%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

from bb_behavior.tracking.pipeline import get_default_pipeline
from tqdm import tqdm_notebook # progress bar
import math
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

Using Theano backend.


In [9]:
import time
import os

def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[-1]


def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))    

In [14]:
datestring = filename_to_datestring('e00_2018-08-19-01-08-13.h264')
timestamp = string_to_timestamp(datestring)
string =  timestamp_to_string(timestamp)

print(datestring)
print(timestamp)
print(string)

2018-08-19-01-08-13
1534633693.0
2018-08-19-01-08-13


In [25]:
# hyperparameters

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.08
confidence_filter_tracks = 0.2
coordinate_scale = 1.0
localizer_threshold="0.55"

#1944

default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

track_dfs = []

counter = 0
for path in glob.glob(os.path.join('../videos/videos_tags/', '*.h264')):  

    print(counter, ": ", path)
    
    try:

        # get the detections
        frame_info, detections = detect_markers_in_video(path,
                                                decoder_pipeline=default_pipeline,
                                                tag_pixel_diameter=tag_pixel_diameter,
                                                n_frames=n_frames,
                                                fps=fps,
                                                progress=progress)

        print(len(detections))
        # get the tracks                                           
        tracks = track_detections_dataframe(detections,
                                            tracker=tracker,
                                            confidence_filter_detections=confidence_filter_detections,
                                            confidence_filter_tracks=confidence_filter_tracks,
                                            coordinate_scale=coordinate_scale)
        t = tracks.copy()
        # get date from filename
        date_string = filename_to_datestring(path)
        timestamp = string_to_timestamp(date_string)

        #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
        grouped = tracks.groupby(['bee_id','track_id'],  as_index=False)['bee_id','xpos','ypos','timestamp']
        df = grouped.aggregate(lambda x: list(x))
        df = df.drop('track_id', 1)

        df['timestamp_video'] = timestamp
        df = df.rename(columns={'timestamp': 'timestamps'})

        # convert bee_id list to single value
        df['bee_id'] = df['bee_id'].apply(lambda x: x[0])

        # only appen if note detections?
        track_dfs.append(df)

        if counter == 4: 
            break
        counter += 1
    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        print(err)
    except Exception as err:
        print(err)
        raise
    
# bind dataframes together
tracks = pd.concat(track_dfs,ignore_index=True)

/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


0 :  ../videos/videos_tags/e00_2018-08-19-01-08-13.h264


Widget Javascript not detected.  It may not be installed or enabled properly.


122


/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.0 when using version 0.19.0. This might lead to breaking code 

'float' object is not subscriptable


TypeError: 'float' object is not subscriptable

In [17]:
# merge tracks of same bee where start and end timestamps are close together
# assume there can not be overlapping tracks
# 1. sort: bee_id, start_time

# calculate start time of track by adding timestamp of track (seconds since start of video)
# to timestamp of video (date)
tracks['start_time'] = tracks['timestamp_video'] + tracks['timestamps'].apply(lambda x: x[0])
tracks['end_time'] = tracks['timestamp_video'] + tracks['timestamps'].apply(lambda x: x[-1])

In [18]:
tracks3 = tracks.copy()

In [19]:
tracks['timestamps'].apply(lambda x: x[-1]) - tracks['timestamps'].apply(lambda x: x[0])

0    13.8
1     2.8
2     1.7
3     2.8
4     1.5
5    14.6
6     0.9
Name: timestamps, dtype: float64

In [20]:
tracks['end_time'] - tracks['start_time']

0    13.8
1     2.8
2     1.7
3     2.8
4     1.5
5    14.6
6     0.9
dtype: float64

In [22]:
t

,id,localizerSaliency,beeID,xpos,ypos,camID,zrotation,timestamp,frameIdx,confidence,track_id,track_confidence,bee_id
0,f11135799832702079370d0c0,0.601259,"[0.9997167, 0.0005463362, 0.9751893, 0.9375251...",1175.158003,100.453682,0,-1.543720,1.6,16,0.813652,0.0,0.668008,1970.0
0,f15396291416620919652d0c0,0.609081,"[0.9987037, 0.0013738275, 0.9999734, 0.9999898...",1275.004221,252.979067,0,0.182082,2.3,23,0.845758,0.0,0.668008,1970.0
0,f17206431796835690422d0c0,0.585542,"[0.99985576, 0.041434586, 0.80112875, 0.999703...",1199.995787,264.929642,0,1.750931,2.8,28,0.339024,0.0,0.668008,1970.0
0,f17403118820136759096d0c0,0.586811,"[0.933081, 0.13542837, 0.9994018, 0.998102, 0....",1101.802593,213.198362,0,2.264336,3.1,31,0.089956,0.0,0.668008,1970.0
0,f10135015133035124699d0c0,0.632266,"[0.9199988, 0.00016194582, 0.999951, 0.9998485...",812.582743,111.473401,0,2.731498,4.0,40,0.824379,1.0,0.528979,1970.0
0,f18021774565449735147d0c0,0.583447,"[0.99914694, 0.0002863109, 0.99987507, 0.99939...",762.665043,175.450041,0,2.465970,4.1,41,0.437863,1.0,0.528979,1970.0
0,f18098907327854750769d0c0,0.647757,"[0.97690547, 0.0020398796, 0.9999411, 0.993224...",738.072926,224.986170,0,2.386785,4.2,42,0.869118,1.0,0.528979,1970.0
0,f9306818199319463948d0c0,0.573610,"[0.00013768673, 0.6283049, 0.9999981, 0.999718...",700.827770,250.133139,0,2.386987,4.3,43,0.227820,1.0,0.528979,1970.0
0,f12570130227525899971d0c0,0.569535,"[0.087643266, 0.773422, 0.99016845, 0.999161, ...",575.859341,223.786334,0,2.783862,4.6,46,0.205970,1.0,0.528979,1970.0
0,f13391862470731687426d0c0,0.699248,"[0.645296, 0.0005341768, 0.99880254, 0.9994539...",512.245116,173.731135,0,-3.029469,4.8,48,0.229836,1.0,0.528979,1970.0


In [21]:
tracks

,bee_id,xpos,ypos,timestamps,timestamp_video,start_time,end_time
0,1926.0,"[912.4305286407471, 974.5363636016846, 1037.02...","[126.9986743927002, 126.95895576477051, 101.19...","[0.0, 0.1, 0.2, 0.3, 0.8, 0.9, 1.2, 1.3, 1.4, ...",1.534634e+09,1.534634e+09,1.534634e+09
1,1926.0,"[463.6847324371338, 513.7263240814209, 576.042...","[151.41355323791504, 151.84414291381836, 139.2...","[17.8, 17.9, 18.0, 18.1, 18.2, 18.3, 18.4, 18....",1.534634e+09,1.534634e+09,1.534634e+09
2,1926.0,"[1200.4261093139648, 1012.0221309661865, 961.8...","[262.57673835754395, 136.45391273498535, 112.3...","[0.0, 0.4, 0.5, 0.9, 1.0, 1.1, 1.2, 1.7]",1.534634e+09,1.534634e+09,1.534634e+09
3,1970.0,"[574.7958679199219, 611.9921283721924, 636.866...","[201.45031547546387, 213.5040740966797, 201.48...","[18.1, 18.2, 18.3, 18.4, 18.5, 18.6, 18.7, 18....",1.534638e+09,1.534638e+09,1.534638e+09
4,1970.0,"[1175.1580028533936, 1275.0042209625244, 1199....","[100.45368194580078, 252.97906684875488, 264.9...","[1.6, 2.3, 2.8, 3.1]",1.534638e+09,1.534638e+09,1.534638e+09
5,1970.0,"[812.58274269104, 762.6650428771973, 738.07292...","[111.47340106964111, 175.45004081726074, 224.9...","[4.0, 4.1, 4.2, 4.3, 4.6, 4.8, 5.7, 9.7, 10.1,...",1.534638e+09,1.534638e+09,1.534638e+09
6,1970.0,"[1186.8357887268066, 1236.4013862609863, 1273....","[139.63128471374512, 113.99038124084473, 127.3...","[19.8, 19.9, 20.0, 20.1, 20.2, 20.7]",1.534638e+09,1.534638e+09,1.534638e+09


In [23]:
tracks = tracks3.copy()
tracks = tracks.sort_values(['bee_id', 'start_time'])

index = 0
tracks2 = tracks.copy()
while(True):
    row = tracks.iloc[index]
    next_row = tracks.iloc[index+1]
    
    # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
    # merge rows means, next_row is deleted
    if (row['bee_id'] == next_row['bee_id']) and ((next_row['start_time'] - row['end_time']) < 10):
        print(next_row['start_time'] - row['end_time'], row['bee_id'], next_row['bee_id'])

        # update the timestamps of nextrow
        t = next_row['start_time'] - row['start_time']
        timestamps = list(np.array(next_row['timestamps']) + t)
        
        tracks.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
        tracks.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
        tracks.at[row.name,'timestamps'] = row['timestamps']+timestamps
        tracks.at[row.name,'end_time'] = next_row['end_time']
        
        tracks.drop(tracks.index[index+1], inplace=True)

    else:
        index += 1
    
    if index == len(tracks) - 1:
        break

4.0 1926.0 1926.0
-0.5999999046325684 1926.0 1926.0
1.6999998092651367 1970.0 1970.0
0.9000000953674316 1970.0 1970.0
1.2000000476837158 1970.0 1970.0


In [24]:
tracks.iloc[0]["timestamps"]

[0.0,
 0.1,
 0.2,
 0.3,
 0.8,
 0.9,
 1.2,
 1.3,
 1.4,
 1.5,
 1.9,
 2.0,
 2.3,
 2.9,
 3.1,
 3.2,
 3.9,
 4.0,
 4.1,
 4.2,
 4.4,
 5.2,
 5.3,
 5.4,
 5.6,
 5.7,
 5.8,
 5.9,
 6.0,
 6.3,
 6.7,
 6.8,
 7.0,
 7.2,
 7.4,
 7.5,
 7.9,
 8.0,
 8.1,
 10.1,
 10.2,
 10.3,
 10.6,
 10.7,
 10.8,
 10.9,
 11.0,
 11.2,
 11.3,
 12.4,
 12.9,
 13.3,
 13.4,
 13.5,
 13.6,
 13.7,
 13.8,
 35.59999995231628,
 35.69999995231628,
 35.799999952316284,
 35.899999952316286,
 35.99999995231629,
 36.09999995231628,
 36.19999995231628,
 36.299999952316284,
 36.399999952316286,
 36.49999995231629,
 36.59999995231628,
 36.69999995231628,
 37.19999995231628,
 37.299999952316284,
 37.59999995231628,
 38.399999952316286,
 20.0,
 20.4,
 20.5,
 20.9,
 21.0,
 21.1,
 21.2,
 21.7]

In [ ]:
tracks = tracks3.copy()
tracks = tracks.sort_values(['bee_id', 'start_time'])
for index, rows in tracks.iterrows():
    row = tracks.iloc[index]
    next_row = tracks.iloc[index+1]
    print(next_row['start_time'] - row['end_time'], row['bee_id'], next_row['bee_id'])

In [ ]:
tracks3

In [ ]:
tracks2

In [ ]:
tracks2

In [ ]:
list(np.array([1,2,3,4,5]))

In [ ]:
tracks.iloc[0]

In [ ]:
# hyperparameters

#e00_2018-08-19-02-20-55

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.08 # 0.08
confidence_filter_tracks = 0.2 #.2
coordinate_scale = 1.0
localizer_threshold="0.50" # 50

#1944


default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

track_dfs = []
counter = 0
for path in ['../videos/videos_tags/e00_2018-08-19-01-08-13.h264']:  

    print(counter, ": ", path)
    
    try:

        # get the detections
        frame_info, detections = detect_markers_in_video(path,
                                                decoder_pipeline=default_pipeline,
                                                tag_pixel_diameter=tag_pixel_diameter,
                                                n_frames=n_frames,
                                                fps=fps,
                                                progress=progress)

        print(len(detections))
        # get the tracks                                           
        tracks = track_detections_dataframe(detections,
                                            tracker=tracker,
                                            confidence_filter_detections=confidence_filter_detections,
                                            confidence_filter_tracks=confidence_filter_tracks,
                                            coordinate_scale=coordinate_scale)
        
        display_tracking_results(path, frame_info, detections, tracks)

    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        print(err)
    except Exception as err:
        print(err)
        raise